# Evaluations
Runs through the different stages of the RAG agent and evaluates the performance of the different stages

In [1]:
import os
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

LIMIT = 100 # set the number of questions to evaluate
results = {}

# temporary output path for saving the data at each stage
tmp_output_path = os.path.join('..', 'eval_data', 'limit_qa_data.json')
results_path = os.path.join('..', 'eval_data', 'results.csv')

## Load the data and prep the dataset

In [2]:
import json

data_dir = os.path.join('..', 'ConvFinQA/data/')

train_data = json.load(open(os.path.join(data_dir, 'train.json')))

Not all the data has qa, so we need to filter out the data that doesn't have qa

In [3]:
qa_data = [
    {
        'id': data['id'],
        'question': data[qa_key]['question'],
        'answer': data[qa_key]['answer']
    }
    for data in train_data
    for qa_key in [k for k in data.keys() if k == 'qa' or k.startswith('qa_')] # account for multiple qa keys
]

In [4]:
qa_data[:5]

[{'id': 'Single_JKHY/2009/page_28.pdf-3',
  'question': 'what was the percentage change in the net cash from operating activities from 2008 to 2009',
  'answer': '14.1%'},
 {'id': 'Single_RSG/2008/page_114.pdf-2',
  'question': 'what was the percent of the growth in the revenues from 2007 to 2008',
  'answer': '1.3%'},
 {'id': 'Single_AAPL/2002/page_23.pdf-1',
  'question': 'what was the percentage change in net sales from 2000 to 2001?',
  'answer': '-32%'},
 {'id': 'Single_UPS/2009/page_33.pdf-2',
  'question': 'what was the difference in percentage cumulative return on investment for united parcel service inc . compared to the s&p 500 index for the five year period ended 12/31/09?',
  'answer': '-26.16%'},
 {'id': 'Double_UPS/2009/page_33.pdf',
  'question': 'what is the roi of an investment in ups in 2004 and sold in 2006?',
  'answer': '-8.9%'}]

In [5]:
# only take the first LIMIT questions
limit_qa_data = qa_data[:LIMIT]

In [6]:
with open(tmp_output_path, 'w') as f:
    json.dump(limit_qa_data, f, indent=4)

## Retrieval
First let's take a look at the retrieval stage.

For this I will look at Recall and Mean Reciprocal Rank (MRR). 

Precision is less relevant here as we always retrieve K documents and there's only 1 relevant document for each question.

In [7]:
from src.nodes import retriever_node

In [8]:
for data in limit_qa_data:
    input_dict = {'question': data['question'], 'metadata_filtering': False}
    retrieved_docs = retriever_node(input_dict)['retrieved_docs']
    data['retrieved_docs'] = retrieved_docs

    retrieved_docs_ids = [doc['id'] for doc in retrieved_docs]
    if data['id'] in retrieved_docs_ids:
        data['retrieved'] = 1
        rank = retrieved_docs_ids.index(data['id']) + 1  # +1 because index starts at 0
        data['reciprocal_rank'] = 1.0 / rank
    else:
        data['retrieved'] = 0
        data['reciprocal_rank'] = 0

In [9]:
# Calculate metrics
retrieved_data = [d for d in limit_qa_data if d['retrieved'] == 1]
num_retrieved = len(retrieved_data)

retrieval_recall = sum(data['retrieved'] for data in limit_qa_data) / len(limit_qa_data)
retrieval_mrr = sum(data['reciprocal_rank'] for data in limit_qa_data) / len(limit_qa_data)
retrieval_mrr_retrieved = sum(data['reciprocal_rank'] for data in retrieved_data) / num_retrieved

print(f"Overall Retrieval Recall: {retrieval_recall:.3f}")
print(f"Overall Retrieval MRR: {retrieval_mrr:.3f}")
print(f"Successful Retrieval MRR: {retrieval_mrr_retrieved:.3f}")

Overall Retrieval Recall: 0.620
Overall Retrieval MRR: 0.268
Successful Retrieval MRR: 0.432


In [10]:
results['overall_retrieval_recall'] = retrieval_recall
results['overall_retrieval_mrr'] = retrieval_mrr
results['successful_retrieval_mrr'] = retrieval_mrr_retrieved

In [11]:
with open(tmp_output_path, 'w') as f:
    json.dump(limit_qa_data, f, indent=4)

### Metatdata Filtering Experiment
Let's run the same epeiment with metadata filtering

In [ ]:
for data in limit_qa_data:
    input_dict = {'question': data['question'], 'metadata_filtering': True}
    mf_retrieved_docs = retriever_node(input_dict)['retrieved_docs']
    data['mf_retrieved_docs'] = mf_retrieved_docs

    mf_retrieved_docs_ids = [doc['id'] for doc in mf_retrieved_docs]
    if data['id'] in mf_retrieved_docs_ids:
        data['mf_retrieved'] = 1
        rank = mf_retrieved_docs_ids.index(data['id']) + 1  # +1 because index starts at 0
        data['mf_reciprocal_rank'] = 1.0 / rank
    else:
        data['mf_retrieved'] = 0
        data['mf_reciprocal_rank'] = 0

In [13]:
# Calculate metrics
mf_retrieved_data = [d for d in limit_qa_data if d['mf_retrieved'] == 1]
num_mf_retrieved = len(mf_retrieved_data)

mf_retrieval_recall = sum(data['mf_retrieved'] for data in limit_qa_data) / len(limit_qa_data)
mf_retrieval_mrr = sum(data['mf_reciprocal_rank'] for data in limit_qa_data) / len(limit_qa_data)
mf_retrieval_mrr_retrieved = sum(data['mf_reciprocal_rank'] for data in mf_retrieved_data) / num_mf_retrieved

print(f"Overall MF Retrieval Recall: {mf_retrieval_recall:.3f}")
print(f"Overall MF Retrieval MRR: {mf_retrieval_mrr:.3f}")
print(f"Successful MF Retrieval MRR: {mf_retrieval_mrr_retrieved:.3f}")

Overall MF Retrieval Recall: 0.580
Overall MF Retrieval MRR: 0.312
Successful MF Retrieval MRR: 0.538


Metadata filter had little effect on retrieval performance. Marginally increasing MRR, but recall was reduced. Metadata filtering will not be used.

## Reranking
Now let's analyse the reranking.

This will also use Recall and MRR.

In [14]:
from src.nodes import reranker_node

import time
from tqdm import tqdm

In [ ]:
for data in tqdm(limit_qa_data, desc="Reranking Documents"):
    input_dict = {'question': data['question'], 'retrieved_docs': data['retrieved_docs']}
    reranked_docs = reranker_node(input_dict)['reranked_docs']
    data['reranked_docs'] = reranked_docs

    reranked_docs_ids = [doc['id'] for doc in reranked_docs]
    if data['id'] in reranked_docs_ids:
        data['reranked'] = 1
        rank = reranked_docs_ids.index(data['id']) + 1  # +1 because index starts at 0
        data['reranked_reciprocal_rank'] = 1.0 / rank
    else:
        data['reranked'] = 0
        data['reranked_reciprocal_rank'] = 0

    # Using the free tier Cohere API you get 10 requests per minute
    # Comment this if you have a paid tier
    time.sleep(6)

In [16]:
# Calculate metrics
reranked_data = [d for d in limit_qa_data if d['reranked'] == 1]
num_reranked = len(reranked_data)

reranking_recall = sum(data['reranked'] for data in limit_qa_data) / len(limit_qa_data)
reranking_mrr = sum(data['reranked_reciprocal_rank'] for data in limit_qa_data) / len(limit_qa_data)
reranked_recall_retrieved = sum(data['reranked'] for data in retrieved_data) / num_retrieved
reranking_mrr_retrieved = sum(data['reranked_reciprocal_rank'] for data in reranked_data) / num_reranked

print(f"Overall Reranking Recall: {reranking_recall:.3f}")
print(f"Overall Reranking MRR: {reranking_mrr:.3f}")
print(f"Successful Reranking Recall: {reranked_recall_retrieved:.3f}")
print(f"Successful Reranking MRR: {reranking_mrr_retrieved:.3f}")

Overall Reranking Recall: 0.540
Overall Reranking MRR: 0.427
Successful Reranking Recall: 0.871
Successful Reranking MRR: 0.790


In [17]:
results['overall_reranking_recall'] = reranking_recall
results['overall_reranking_mrr'] = reranking_mrr
results['successful_reranking_recall'] = reranked_recall_retrieved
results['successful_reranking_mrr'] = reranking_mrr_retrieved

In [18]:
with open(tmp_output_path, 'w') as f:
    json.dump(limit_qa_data, f, indent=4)

## LLM Correctness
In other RAG systems where the answer is expected as a paragraph or may use multiple sources, LLM faithfulness can be  agood metric. In our case with ConvFinQA, the answer is expected as a single number, and therefore a more binary metric is more appropriate.

In [20]:
limit_qa_data = json.load(open(tmp_output_path))

In [21]:
from src.nodes import answer_question_node

import re
import math

In [22]:
def clean_number(text):
    """
    Extract and normalize number from text, removing symbols and units.
    Returns float or None if no number found.
    """
    
    # Remove common units and words
    text = text.lower()
    text = re.sub(r'%|percent|million|thousand|billion|m|k|b|\$', '', text)

    # handle yes/no answer present in the data
    if text in ['yes', 'no']:
        return text
    
    # Remove commas from numbers (e.g., "25,000" -> "25000")
    text = re.sub(r'(\d),(\d)', r'\1\2', text)
    
    # Find numbers (including decimals and negatives)
    numbers = re.findall(r'-?\d*\.?\d+', text)
    
    if numbers:
        return float(numbers[0])  # Return first number found
    return None

In [23]:
for i, data in tqdm(enumerate(limit_qa_data), desc="Answering Questions"):
    input_dict = {'question': data['question'], 'reranked_docs': data['reranked_docs'], 'chat': False}
    raw_rag_answer = answer_question_node(input_dict)['answer']
    data['raw_rag_answer'] = raw_rag_answer

    # clean the answers for comparison
    cleaned_original_answer = clean_number(data['answer'])
    cleaned_rag_answer = clean_number(raw_rag_answer)
    data['cleaned_rag_answer'] = cleaned_rag_answer

    # now do the comparison
    correct = False
    # accounts for 'yes' and 'no' answers and direct matches
    if cleaned_rag_answer == cleaned_original_answer:
        correct = True
    elif isinstance(cleaned_rag_answer, float) and isinstance(cleaned_original_answer, float):
        # account for the fact that the original answer is provided with no decimal places (e.g. 17%) but I provide more precide value (e.g. 17.1% or 16.9%)
        # also considers absolute matches the same to account percetnage change questions
        if cleaned_original_answer.is_integer():
            int_rag = int(cleaned_rag_answer)
            rounded_rag = round(cleaned_rag_answer)
            if abs(cleaned_original_answer) in (abs(int_rag), abs(rounded_rag)):
                correct = True
        # standard case
        else:
            if math.isclose(abs(cleaned_rag_answer), abs(cleaned_original_answer), rel_tol=0.01, abs_tol=0.05): # 0.01 seems appropriate for this data to account for LLM rounding errors
                correct = True
    
    data['correct'] = 1.0 if correct else 0.0

    print("-"*70)
    print(f"Document {i+1}: {data['id']}")
    print(f"Question {i+1}: {data['question']}")
    print(f"Original Answer: {data['answer']}")
    print(f"RAG Answer: {raw_rag_answer}")
    print(f"Cleaned Original Answer: {cleaned_original_answer}")
    print(f"Cleaned RAG Answer: {cleaned_rag_answer}")
    print(f"Correct: {correct}")
    print("-"*70)
    print("\n")

correct_count = sum(data['correct'] for data in limit_qa_data)
accuracy = correct_count / len(limit_qa_data)
print(f"Correct Count: {correct_count}")
print(f"Accuracy: {accuracy:.3f}")

Answering Questions: 0it [00:00, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Answering Questions: 1it [00:01,  1.52s/it]

----------------------------------------------------------------------
Document 1: Single_JKHY/2009/page_28.pdf-3
Question 1: what was the percentage change in the net cash from operating activities from 2008 to 2009
Original Answer: 14.1%
RAG Answer: 14.2%
Cleaned Original Answer: 14.1
Cleaned RAG Answer: 14.2
Correct: True
----------------------------------------------------------------------




Answering Questions: 2it [00:03,  1.73s/it]

----------------------------------------------------------------------
Document 2: Single_RSG/2008/page_114.pdf-2
Question 2: what was the percent of the growth in the revenues from 2007 to 2008
Original Answer: 1.3%
RAG Answer: 16.0%
Cleaned Original Answer: 1.3
Cleaned RAG Answer: 16.0
Correct: False
----------------------------------------------------------------------




Answering Questions: 3it [00:05,  1.84s/it]

...

----------------------------------------------------------------------
Document 98: Single_AAL/2015/page_51.pdf-3
Question 98: by how much did american airlines group inc . common stock out preform the s&p 500 index over the 4 year period?
Original Answer: 48%
RAG Answer: 62%
Cleaned Original Answer: 48.0
Cleaned RAG Answer: 62.0
Correct: False
----------------------------------------------------------------------




Answering Questions: 99it [10:34,  6.32s/it]

----------------------------------------------------------------------
Document 99: Double_LKQ/2009/page_77.pdf
Question 99: what was the percentage change in rental expense for operating leases from 2007 to 2008?
Original Answer: 84%
RAG Answer: 84.21%
Cleaned Original Answer: 84.0
Cleaned RAG Answer: 84.21
Correct: True
----------------------------------------------------------------------




Answering Questions: 100it [10:39,  6.39s/it]

----------------------------------------------------------------------
Document 100: Double_LKQ/2009/page_77.pdf
Question 100: what was the percentage change in rental expense for operating leases from 2008 to 2009?
Original Answer: 17%
RAG Answer: 16.73%
Cleaned Original Answer: 17.0
Cleaned RAG Answer: 16.73
Correct: True
----------------------------------------------------------------------


Correct Count: 44.0
Accuracy: 0.440


In [24]:
with open(tmp_output_path, 'w') as f:
    json.dump(limit_qa_data, f, indent=4)

In [25]:
results['accuracy'] = accuracy

In [26]:
import pandas as pd

results_df = pd.DataFrame({
    'Metric': results.keys(),
    'Value': results.values()
})

results_df.to_csv(results_path, index=False)

In [28]:
display(results_df)

,Metric,Value
0,overall_retrieval_recall,0.620000
1,overall_retrieval_mrr,0.267619
2,successful_retrieval_mrr,0.431644
3,overall_reranking_recall,0.540000
4,overall_reranking_mrr,0.426667
5,successful_reranking_recall,0.870968
6,successful_reranking_mrr,0.790123
7,accuracy,0.440000
